In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
# TO USE:
# Uncomment three !pip install lines
# Run this cell ONLY
# Runtime -> Restart session
# Comment three !pip install lines
# Run all

# Andy can push

# Pandas 1.5.3
# Numpy 1.26.3
# Json 2.0.9
# NLTK (Natural Language Toolkit) 3.8.1
# Gensim 4.3.2
# Spacy 3.6.1
# PyLDAvis 3.4.0
# re (removing characters) 2.2.1
# clean from cleantext
# from prompt_toolkit.completion import word_completer
# from pprint import pprint
# import matplotlib.pyplot as plt


%pip install pyLDAvis
%pip install --user -U nltk
%pip install -U spacy
%pip install "pandas==1.5.3"

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.2/11.6 MB 2.9 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/11.6 MB 6.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/11.6 MB 8.5 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.6 MB 8.2 MB/s eta 0:00:02
   ------- -------------------------------- 2.2/11.6 MB 8.7 MB/s eta 0:00:02
   --------- ------------------------------ 2.7/11.6 MB 8.6 MB/s eta 0:00:02
   ----------- ---------------------------- 3.3/11.6 MB 9.2 MB/s eta 0:00:01
   ------------- -------------------------- 3.8/11.6 MB 9.4 MB/s eta 0:00:01
   --------------- ------------------------ 4.4/11.6 MB 9.6 MB/s eta 0:00:01
   ---------------- ----------------------- 4.8/11.6 MB 9.6 MB/s eta 0:00:01
   ------------------- -------------------- 5.5/11.6 MB 10.1 MB/s eta 0:00:01
   ---------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


In [2]:
#

import numpy as np
import json
import glob
import nltk
import pandas as pd #to work with csv files

# pd.__version__ = 2.0.0
print(pd.__version__)

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

1.5.3


In [3]:
import os
#The function that loops through the data to check for optimal parameters to input into the LDA model would take over 16 hours to run,
#after a while the runtime disconnects, which makes the progress start from 0.
#so the purpose of this function is to make sure the runtime doesnt disconnect, while training for optimal params to input to lda model.

#def check_and_reconnect_drive():
    #try:
        # Check if Google Drive is still connected
     #   os.listdir('/content/drive')
  #  except:
        # If not, reconnect it
  #      from google.colab import drive
  #      drive.mount('/content/drive', force_remount=True)

# Then call this function every so often in your main script
#check_and_reconnect_drive()

In [4]:
nltk.download('stopwords')

stopwords = stopwords.words("english") #get stopwords from nltk

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#stopwords.extend(['hi', 'anymore', 'years', 'ago', 'almost', 'somehow', 'remember', 'wants', 'either', 'enough', 'much', 'making', 'simple', 'things', 'add', 'actual', 'say', 'etc', 'anything', 'days', 'long', 'another', 'ever', 'yes', 'th', 'ice', 'mightymouse', 'should', 'obviously', 'even', 'lol', 'stuff', 'never', 'old', 'yr', 'bit', 'like', 'aka', 'nonsense', 'ok', 'absolutely', 'op', 'weird', 'ass', 'fully', 'get', 'would', 'definitely', 'especially', 'still', 'else', 'something', 'often', 'kind', 'let', 'lot', 'able', 'non', 'pg', 'nice', 'trad', 'keep', 'massive', 'quickly', 'be', 'common', 'tv', 'fuckin', 'im', 'could', 'also', 'silly', 'ie', 'seem', 'seems', 'fucked'])#Add custom stop words
stopwords.extend(['r/parenting', 'r/roblox', 'get'])#Add reddit paths

In [6]:
def wordcounts(text1):
  fdist1 = FreqDist(text1)
  print(fdist1)
  fdist1.most_common(50)

In [7]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

In [8]:
def load_data(file):
  data = pd.read_csv(file)
  return data

In [9]:
data = load_data('../Dataset/cumulative5.csv')
filtered_data = data.drop(columns=['subreddit','date','timestamp'], axis=1)
data = filtered_data.replace('\n', ' ', regex=True)
# data.head(5)

In [10]:
# data[data.text.str.contains('I am a bot')]

In [11]:
import re

data = data.loc[data.text.str.contains('I am a bot') == False, :]

# Remove punctuation
# data['text'] = data['text'].map(lambda x: re.sub("[,\.!?/@#$^&*-_=+()%]", "", x))
data['text'] = data['text'].replace(r'http\S+', '', regex=True)

#data['text'] = data['text'].replace(r'a'+ u'\u0301'+ 'U+20AC'+ 'U+2122', "'" , regex=True)

data['text'] = data['text'].map(lambda x: re.sub('[%$:,\.!?/-]', ' ', x))
data['text'] = data['text'].map(lambda x: re.sub('\(', ' ', x))
data['text'] = data['text'].map(lambda x: re.sub('\)', ' ', x))
data['text'] = data['text'].replace(r'&\S+', '', regex=True)
data['text'] = data['text'].replace('\"', '', regex=True)

# # Convert the titles to lowercase
data['text'] = data['text'].map(lambda x: x.lower())

pattern = r'[0-9]'
data['text'] = data['text'].map(lambda x: re.sub(pattern, ' ', x))
data['text'] = data['text'].map(lambda x: re.sub(' +', ' ', x))

data.shape

(12075, 1)

In [12]:
data.columns

Index(['text'], dtype='object')

In [13]:
data.head(5)

,text
0,live chat hi does anyone know if i can access ...
1,forgot pin i created my parental pin almost ye...
2,roblox rant this game…honestly either not enou...
3,you're not alone that's all i have to say
4,i even contacted support and they were useless


In [14]:
data[data.text.str.contains('I am a bot')]

,text


In [15]:
data['text'][51]

'definitely not for year olds i hope at the very least chat is not enabled there is a lot of random violence the best option is for them to only join a private server which are free and easy to set up with a couple of clicks for any game that includes only just the kids in the daycare '

In [16]:
!pip install clean-text
from cleantext import clean

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [17]:
data = [clean(d, no_emoji=True) for d in data['text']]

In [18]:
data = remove_stopwords(data)

In [19]:
resultx = []
for d in data:
  resultx.append(' '.join(d))

#resultxtokens = resultx.split()


In [20]:
resultx[:5]

['live chat hi anyone know access live chat history child account tried contacting roblox heard back',
 'forgot pin created parental pin almost years ago somehow remember son accidentally blocked brother tried email roblox reset pin son upset wants play brother reset without pin anyone else happen',
 'roblox rant gamehonestly either enough protection young kids much protection making game useless chat simple things add parental game controls parents approve friend requests outgoing incoming experience chats stored parent accessible actual inappropriate text controls anyone walk kid roblox say private parts etc friends experience chat history reported users missing anything specifically pissed find days long experience chat child another user extremely inappropriate zero way access anything regarding going honest open conversation child',
 'alone say',
 'even contacted support useless']

In [21]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download('punkt')
words = [word_tokenize(i) for i in resultx]
words2= [num for elem in words for num in elem]
#words = nltk.tokenize.word_tokenize(resultx)

#wordcounts(words2)
fdist1 = FreqDist(words2)
#print(fdist1)
fdist1.most_common(50)


#>>> # flatten a list using a listcomp with two 'for'
#>>> vec = [[1,2,3], [4,5,6], [7,8,9]]
#>>> [num for elem in vec for num in elem]
#[1, 2, 3, 4, 5, 6, 7, 8, 9]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('kids', 3958),
 ('like', 3569),
 ('roblox', 3167),
 ('games', 2714),
 ('time', 2643),
 ('game', 2527),
 ('play', 2502),
 ('would', 2230),
 ('kid', 2027),
 ('also', 1874),
 ('old', 1811),
 ('think', 1806),
 ('money', 1766),
 ('know', 1712),
 ('one', 1635),
 ('make', 1618),
 ('year', 1554),
 ('things', 1488),
 ('want', 1420),
 ('good', 1391),
 ('people', 1390),
 ('even', 1347),
 ('really', 1333),
 ('phone', 1333),
 ('something', 1299),
 ('child', 1255),
 ('much', 1246),
 ('let', 1211),
 ('way', 1203),
 ('daughter', 1187),
 ('son', 1185),
 ('friends', 1153),
 ('school', 1085),
 ('playing', 1080),
 ('parents', 1076),
 ('need', 1067),
 ('age', 1011),
 ('still', 1005),
 ('could', 1002),
 ('see', 964),
 ('use', 960),
 ('go', 946),
 ('day', 943),
 ('going', 935),
 ('thing', 933),
 ('back', 928),
 ('got', 920),
 ('lot', 902),
 ('say', 900),
 ('stuff', 827)]

In [22]:
result = []
for d in data:
  result.append(' '.join(d))
# result

In [23]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# data = filtered_data.review_text_processed.values.tolist()
data_words = list(sent_to_words(result))

# prints the first 30 words of the first tokenized review from the data_words list.
# print(data_words[:1][0][:30])

In [24]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [25]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [26]:
# # NO BIGRAMS
# !python -m spacy download en_core_web_sm
# import spacy

# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [27]:
# BIGRAMS DATA
!python -m spacy download en_core_web_sm
import spacy

# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 165.2 kB/s eta 0:01:18
     --------------------------------------- 0.0/12.8 MB 281.8 kB/s eta 0:00:46
      --------------------------------------- 0.2/12.8 MB 1.1 MB/s eta 0:00:13
     - -------------------------------------- 0.5/12.8 MB 2.4 MB/s eta 0:00:06
     --- ------------------------------------ 1.0/12.8 MB 3.8 MB/s eta 0:00:04
     ---- ----------------------------------- 1.5/12.8 MB 4.9 MB/s eta 0:00:03
     ------ --------------------------------- 2.2/12.8 MB 6.0 MB/s eta 0:00:02
     -------- ------------------------------- 2.7/12.8 MB 6.3 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 6.7 MB/s eta 0:00:02
     ----------- ---------------------------- 3.6/12.8 MB 7.2 MB/s eta 0:00:02
     ------------ --------------------------- 4.0/12.8 MB 7.6 MB

In [28]:
print(type(data_lemmatized))

<class 'list'>


Remove Duplicate Words (TO REMOVE)

In [29]:
# from collections import Counter
# def unique(d):
#   UniqW = Counter(d)
#   result = [*UniqW]
#   # corpus = ' '.join(UniqW.keys())
#   return result

In [30]:
# single_data = []
# for d in data_lemmatized:
#   for word in d:
#     single_data.append(word)

In [31]:
# unique_data = [unique(d) for d in data_lemmatized]
# # unique_data = unique(single_data)

In [32]:
# print(type(unique_data))

In [33]:
# unique_data

In [34]:
from prompt_toolkit.completion import word_completer
id2word = corpora.Dictionary(data_lemmatized)
# id2word = corpora.Dictionary(bag_of_words)

corpus = []
for text in data_lemmatized:
  new = id2word.doc2bow(text)
  corpus.append(new)

print(corpus[0])

print(data_words[:1][0][:30])
# print(word_completer)

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1)]
['live', 'chat', 'hi', 'anyone', 'know', 'access', 'live', 'chat', 'history', 'child', 'account', 'tried', 'contacting', 'roblox', 'heard', 'back']


In [35]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=5,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True,
                                       alpha=0.01,
                                       eta=0.61)

In [36]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.153658  0.068794       1        1  42.397754
2      0.071214  0.150595       2        1  36.717472
0      0.093799 -0.082058       3        1  15.300491
1     -0.182409  0.035916       4        1   3.531978
4     -0.136262 -0.173246       5        1   2.052305, topic_info=        Term         Freq        Total Category  logprob  loglift
41      game  5052.000000  5052.000000  Default  30.0000  30.0000
83     money  1378.000000  1378.000000  Default  29.0000  29.0000
21      play  3762.000000  3762.000000  Default  28.0000  28.0000
9     roblox  2363.000000  2363.000000  Default  27.0000  27.0000
1    account   552.000000   552.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
223      set    18.418480   540.787669   Topic5  -5.7877   0.5065
115     scam    15.438593   250.106328   Topic5  -5.9642   1.1012
75   support    14.104516   157.961667   Topic5  -6.0546   1.4703
118     send    14.027611   193.382868   Topic5  -6.0600   1.2626
300      app    13.859931   352.397147   Topic5  -6.0721   0.6504

[381 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1         1  0.391149  account
1         2  0.005433  account
1         3  0.387527  account
1         5  0.215494  account
30        1  0.493659      add
...     ...       ...      ...
234       4  0.010192     year
1130      1  0.991520  youtube
1130      2  0.002687  youtube
1130      3  0.002687  youtube
1130      5  0.002687  youtube

[1160 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2, 5])

In [37]:
from pprint import pprint
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.031*"money" + 0.018*"buy" + 0.015*"make" + 0.015*"game" + 0.013*"spend" + '
  '0.011*"want" + 0.008*"kid" + 0.008*"pay" + 0.008*"get" + 0.008*"thing"'),
 (1,
  '0.012*"love" + 0.008*"bluey" + 0.006*"wear" + 0.006*"lol" + 0.005*"girl" + '
  '0.005*"make" + 0.004*"old" + 0.004*"show" + 0.003*"theme" + 0.003*"party"'),
 (2,
  '0.018*"time" + 0.014*"kid" + 0.013*"go" + 0.010*"phone" + 0.009*"need" + '
  '0.009*"take" + 0.009*"say" + 0.009*"get" + 0.009*"day" + 0.008*"make"'),
 (3,
  '0.035*"game" + 0.029*"play" + 0.023*"kid" + 0.016*"roblox" + 0.009*"know" + '
  '0.009*"old" + 0.009*"make" + 0.008*"friend" + 0.008*"think" + '
  '0.008*"thing"'),
 (4,
  '0.020*"account" + 0.009*"email" + 0.009*"password" + 0.008*"roblox" + '
  '0.007*"use" + 0.005*"purchase" + 0.005*"ban" + 0.004*"free" + '
  '0.004*"setting" + 0.004*"model"')]


In [38]:
from gensim.models import CoherenceModel

# Compute Coherence Score

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, corpus=corpus, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5254998879902841


In [39]:
# Get the topic distribution for each document
document_topics = lda_model.get_document_topics(corpus)

# Filter reviews that have topic 1 as their main topic
reviews_in_topic_1 = [idx for idx, topics in enumerate(document_topics) if max(topics, key=lambda x: x[1])[0] == 1]

i = 0
# Print the reviews
for idx in reviews_in_topic_1:
    i += 1
    print(' '.join(data_words[idx]))

print(i)

bloxburg stats
stripping
yeah snowy strange lol
worth replying take downvote buzz
jokes lolol
farted
welcome capitalism
guess stuck knee deep roblox capitalism crap
lol
lit totally relate
check black mirror arkangel episode
hated outdoors child asthmatic allergic everything constantly getting bitten bugs like getting dirty anxiety thing loved creative pretend play best playdates usually involved sort small plastic barbie bratz polly pocket hot wheels littlest pet shop dudes lego perhaps combination thereof polly spider man often dated vaguely polly sized spider man action figure mix good ol open world storytelling got older bake dates make cupcakes cookies banana bread go upstairs experiment makeup ate gossiped kind relationship trying build little niblings one based idea indoorsy synonymous screens time
lol
wow classic hardcore
love bed wars lol
remember parking runescape telling friend eating klondike bar confused half word censored lol
hell come lol
yeah business raising losers ener

In [40]:

def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaModel(corpus=corpus,
                                       id2word=dictionary,
                                       num_topics=k,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       alpha=a,
                                       eta=b,
                                       per_word_topics=True)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    return coherence_model_lda.get_coherence()

#data_words

In [41]:
# import numpy as np
# import tqdm
# grid = {}
# grid['Validation_Set'] = {}

#  # Topics range
# min_topics = 2
# max_topics = 11
# step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
# # Alpha parameter
# #A list of alpha values to be tested.
# #It includes a range of values from 0.01 to 1 with a step size of 0.3, as well as two special values: 'symmetric' and 'asymmetric'.
# alpha = list(np.arange(0.01, 1, 0.3))
#  #assumes that all documents are equally likely to be a mixture of topics, resulting in a uniform distribution of topics across documents. So it maps rreviews to each topic evenly
#  #It makes the model assume that every document should have roughly the same proportion of topics.
# # alpha.append('symmetric')
# # #allows for more variability in document-topic proportions.
# # #It doesn't enforce a uniform distribution and allows documents to have varying degrees of focus on different topics.
# alpha.append('asymmetric')

# # # Beta parameter
# # #A list of beta values to be tested. It follows a similar pattern to alpha, including a range and 'symmetric'.
# beta = list(np.arange(0.01, 1, 0.3))
# beta.append('symmetric')

# # # Validation sets:
# # # We have the corpus sets, which contains 2 versions of the corpus, 15% and 20%
# num_of_docs = len(corpus)

# # #A list containing two versions of the 15% corpus, one with 20% of the documents and another with 15% of the documents.
# corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.20)),
#                gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.15))]

# corpus_title = ['20% Corpus', '15% Corpus']

# #Stores the results of the LDA model with different hyperparameters.
# model_results = {'Validation_Set': [],
#                   'Topics': [],
#                   'Alpha': [],
#                   'Beta': [],
#                   'Coherence': []
#                  }

#  # Can take a long time to run
# if 1 == 1:
#    #progress bar
#     pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))

#      # iterate through validation corpus sets
#     for i in range(len(corpus_sets)):
#          # iterate through number of topics
#         for k in topics_range:
#              # iterate through alpha values
#             for a in alpha:
#                  # iterare through beta values
#                 for b in beta:
#                      #reconnect the runtime
#                    # check_and_reconnect_drive()
#                      # get the coherence score for the given parameters
#                     cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word,
#                                                   k=k, a=a, b=b)

#                      # Save the model results
#                     model_results['Validation_Set'].append(corpus_title[i])
#                     model_results['Topics'].append(k)
#                     model_results['Alpha'].append(a)
#                     model_results['Beta'].append(b)
#                     model_results['Coherence'].append(cv)
#                     pbar.update(1)
#     pd.DataFrame(model_results).to_csv('../Dataset/lda_tuning_results.csv', index=False)
#      # pd.DataFrame(model_results).to_csv('/content/drive/My Drive/Classes/lda_tuning_results.csv', index=False)
#     pbar.close()

In [61]:
import os


print(os.getcwd())

c:\Users\Owner\Favorites\LDA_Analysis\Code


In [42]:
# if 1==1:
# # pd.DataFrame(model_results).to_csv('./results/lda_tuning_results.csv', index=False)
#     pd.DataFrame(model_results).to_csv('/content/drive/My Drive/Classes/lda_tuning_results.csv', index=False)
#     pbar.close()

In [43]:
# import matplotlib.pyplot as plt
# import pandas as pd

# # Load the model results from the CSV file
# model_results_df = pd.read_csv('/content/lda_tuning_results.csv')

# # Filter the results for the 75% Corpus (you can change this if needed)
# # filtered_results = model_results_df[model_results_df['Validation_Set'] == '20% Corpus']

# # Group the results by the number of topics (Topics) and calculate the mean coherence score
# coherence_by_topics = model_results_df.groupby('Topics')['Coherence'].mean()

# # Plot the coherence score against the number of topics
# plt.figure(figsize=(12, 6))
# plt.plot(coherence_by_topics.index, coherence_by_topics.values, marker='o', linestyle='-')
# plt.xlabel('Number of Topics')
# plt.ylabel('Coherence Score (C_v)')
# plt.title('Coherence Score vs. Number of Topics')
# plt.grid(True)
# plt.show()

In [44]:
# data_topic_3 = model_results_df[model_results_df['Topics'] == 2]
# max_coherence = np.array(data_topic_3['Coherence'])
# print(max_coherence[4])
# # print(np.argmax(max_coherence))

##Visualizing the Data


In [45]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.153658  0.068794       1        1  42.406643
2      0.071214  0.150595       2        1  36.710621
0      0.093799 -0.082058       3        1  15.301080
1     -0.182409  0.035916       4        1   3.529416
4     -0.136262 -0.173246       5        1   2.052239, topic_info=        Term         Freq        Total Category  logprob  loglift
41      game  5053.000000  5053.000000  Default  30.0000  30.0000
83     money  1378.000000  1378.000000  Default  29.0000  29.0000
21      play  3763.000000  3763.000000  Default  28.0000  28.0000
9     roblox  2363.000000  2363.000000  Default  27.0000  27.0000
1    account   552.000000   552.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
223      set    18.417884   540.777873   Topic5  -5.7877   0.5066
115     scam    15.438093   250.119471   Topic5  -5.9642   1.1011
75   support    14.104059   157.953242   Topic5  -6.0546   1.4704
118     send    14.027157   193.376208   Topic5  -6.0600   1.2626
300      app    13.859482   352.398640   Topic5  -6.0721   0.6504

[381 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1         1  0.391114  account
1         2  0.005432  account
1         3  0.387493  account
1         5  0.215475  account
30        1  0.493621      add
...     ...       ...      ...
234       4  0.010192     year
1130      1  0.991316  youtube
1130      2  0.002686  youtube
1130      3  0.002686  youtube
1130      5  0.002686  youtube

[1159 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2, 5])

In [46]:
lda_model_tuned = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=2,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True,
                                       alpha=0.31,
                                       eta='symmetric')

In [47]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tuned, corpus, id2word, mds='mmds', R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.005137 -0.090663       1        1  71.337771
0      0.005137  0.090663       2        1  28.662229, topic_info=       Term         Freq        Total Category  logprob  loglift
41     game  4985.000000  4985.000000  Default  30.0000  30.0000
83    money  1382.000000  1382.000000  Default  29.0000  29.0000
253     buy   908.000000   908.000000  Default  28.0000  28.0000
9    roblox  2429.000000  2429.000000  Default  27.0000  27.0000
49     make  2785.000000  2785.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
74     even   357.291160  1289.077291   Topic2  -5.4591  -0.0335
71      way   354.487604  1310.732381   Topic2  -5.4670  -0.0581
62      say   356.396986  1906.994828   Topic2  -5.4616  -0.4276
3     child   350.484065  1773.762584   Topic2  -5.4784  -0.3720
140     see   339.386030  1366.805831   Topic2  -5.5106  -0.1435

[176 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1         1  0.108486   account
1         2  0.891531   account
603       1  0.993290  activity
603       2  0.007700  activity
794       1  0.003481     adopt
...     ...       ...       ...
886       2  0.004213      wife
330       1  0.702695      work
330       2  0.297681      work
234       1  0.927623      year
234       2  0.072429      year

[244 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [48]:
# Get the topic distribution for each document
document_topics = lda_model.get_document_topics(corpus)
print(document_topics)


In [49]:
# Filter reviews that have topic 1 as their main topic
reviews_in_topic_1 = [idx for idx, topics in enumerate(document_topics) if max(topics, key=lambda x: x[1])[0] == 1]

# Get the number of reviews in topic 1
num_reviews_in_topic_1 = len(reviews_in_topic_1)

print(f'There are {num_reviews_in_topic_1} reviews in topic 1.')


There are 529 reviews in topic 1.


In [50]:
#import pandas as pd
#reviews_in_topic_1 = [idx for idx, topics in enumerate(document_topics) if max(topics, key=lambda x: x[1])[0] == 1]

#mixture = [dict(lda_model[x]) for x in corpus]
pd.DataFrame(reviews_in_topic_1).to_csv("topic_mixture.csv")

In [51]:
print(max(document_topics))

[(4, 0.9976535)]


In [52]:
# Filter reviews that have topic 5 as their main topic
reviews_in_topic_5 = [idx for idx, topics in enumerate(document_topics) if max(topics, key=lambda x: x[1])[0] == 5]

# Get the number of reviews in topic 5
num_reviews_in_topic_5 = len(reviews_in_topic_5)

print(f'There are {num_reviews_in_topic_5} reviews in topic 5.')

There are 0 reviews in topic 5.


In [53]:
datax= load_data('../Dataset/cumulative5.csv')
filtered_data = datax.drop(columns=['subreddit','date','timestamp'], axis=1)
datax= filtered_data.replace('\n', ' ', regex=True)
import re

datax= datax.loc[datax.text.str.contains('I am a bot') == False, :]

# Remove punctuation
# data['text'] = data['text'].map(lambda x: re.sub("[,\.!?/@#$^&*-_=+()%]", "", x))
datax['text'] = datax['text'].replace(r'http\S+', '', regex=True)

#data['text'] = data['text'].replace(r'a'+ u'\u0301'+ 'U+20AC'+ 'U+2122', "'" , regex=True)

datax['text'] = datax['text'].map(lambda x: re.sub('[%$:,\.!?/-]', ' ', x))
datax['text'] = datax['text'].map(lambda x: re.sub('\(', ' ', x))
datax['text'] = datax['text'].map(lambda x: re.sub('\)', ' ', x))
datax['text'] = datax['text'].replace(r'&\S+', '', regex=True)
datax['text'] = datax['text'].replace('\"', '', regex=True)

# # Convert the titles to lowercase
datax['text'] = datax['text'].map(lambda x: x.lower())

pattern = r'[0-9]'
datax['text'] = datax['text'].map(lambda x: re.sub(pattern, ' ', x))
datax['text'] = datax['text'].map(lambda x: re.sub(' +', ' ', x))

datax.shape

(12075, 1)

In [54]:
datax.head(5)

,text
0,live chat hi does anyone know if i can access ...
1,forgot pin i created my parental pin almost ye...
2,roblox rant this game…honestly either not enou...
3,you're not alone that's all i have to say
4,i even contacted support and they were useless


In [55]:
datax['text'][3]
type(datax['text'][3])

resultx2 = []
for d in datax['text']:
  resultx2.append(d)

#resultx2.head[5]

In [56]:
# Get the comments without weird spaces


i = 0
# Print the reviews
#filex=[""]
id=[" "]
textx=[" "]
comment=""
for i in datax['text']:
    #i += 1
    #print(idx)
    #print(' '.join(data_words[ix]))
    id.append(i)
    textx.append(i)
    # create a dictionary with the three lists
dict = {'ID': id,'Text':textx}

# create a Pandas DataFrame from the dictionary
df = pd.DataFrame(dict)

# write the DataFrame to a CSV file
df.to_csv('Data.csv')

#pd.DataFrame(filex).to_csv("topic_mixture1.csv")
#print(i)


In [57]:
# Get the topic distribution for each document
document_topics = lda_model.get_document_topics(corpus)

# Filter reviews that have topic 1 as their main topic
reviews_in_topic_1_tuned = [idx for idx, topics in enumerate(document_topics) if max(topics, key=lambda x: x[1])[0] == 1]

i = 0
# Print the reviews
filex=[""]
id=[" "]
textx=[" "]
comment=""
for idx in reviews_in_topic_1:
    i += 1
    #print(idx)
    #print(' '.join(data_words[idx]))
    filex.append(data_words[idx])
    id.append(idx)
    textx.append(resultx2[idx])
    # create a dictionary with the three lists
dict = {'top words': filex, 'ID': id,'Text':textx}

# create a Pandas DataFrame from the dictionary
df = pd.DataFrame(dict)

# write the DataFrame to a CSV file
df.to_csv('EmployeeData.csv')

#pd.DataFrame(filex).to_csv("topic_mixture1.csv")
#print(i)


In [58]:
# Get the topic distribution for each document
document_topics_tuned = lda_model_tuned.get_document_topics(corpus)

# Filter reviews that have topic 1 as their main topic
reviews_in_topic_1_tuned = [idx for idx, topics in enumerate(document_topics) if max(topics, key=lambda x: x[1])[0] == 1]

i = 0
# Print the reviews
for idx in reviews_in_topic_1_tuned:
    i += 1
    print(' '.join(data_words[idx]))

print(i)

bloxburg stats
stripping
yeah snowy strange lol
worth replying take downvote buzz
jokes lolol
farted
welcome capitalism
guess stuck knee deep roblox capitalism crap
lol
lit totally relate
check black mirror arkangel episode
hated outdoors child asthmatic allergic everything constantly getting bitten bugs like getting dirty anxiety thing loved creative pretend play best playdates usually involved sort small plastic barbie bratz polly pocket hot wheels littlest pet shop dudes lego perhaps combination thereof polly spider man often dated vaguely polly sized spider man action figure mix good ol open world storytelling got older bake dates make cupcakes cookies banana bread go upstairs experiment makeup ate gossiped kind relationship trying build little niblings one based idea indoorsy synonymous screens time
lol
wow classic hardcore
love bed wars lol
remember parking runescape telling friend eating klondike bar confused half word censored lol
hell come lol
yeah business raising losers ener

In [59]:
# Filter reviews that have topic 1 as their main topic
reviews_in_topic_2_tuned = [idx for idx, topics in enumerate(document_topics) if max(topics, key=lambda x: x[1])[0] == 2]

i = 0
# Print the reviews
filex=[""]
id=[" "]
textx=[" "]
comment=""
for idx in reviews_in_topic_2_tuned:
    i += 1
    #print(idx)
    #print(' '.join(data_words[idx]))
    filex.append(data_words[idx])
    id.append(idx)
    textx.append(resultx2[idx])
    # create a dictionary with the three lists
dict = {'top words': filex, 'ID': id,'Text':textx}

# create a Pandas DataFrame from the dictionary
df = pd.DataFrame(dict)

# write the DataFrame to a CSV file
df.to_csv('EmployeeData2.csv')

#pd.DataFrame(filex).to_csv("topic_mixture1.csv")
#print(i)
